# Sentiment Analysis

In this notebook we will show how we build a model to give a polarity to the tweets
which contains about famous comapany

In [10]:
from collections import Counter
import numpy as np
import pandas as pd
from imblearn.datasets import make_imbalance
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
import re
from langdetect import detect
from bs4 import BeautifulSoup

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/tom/.conda/envs/pythonProject/lib/python3.10/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib\n  Referenced from: /Users/tom/.conda/envs/pythonProject/lib/python3.10/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file)"]


We have two different dataset of financial tweets to train and test our model, one is downloaded from Kaggle, the other is composed by real tweest screaped from tweeter and where we have put the label of the class by hand.


In [ ]:
from preprocessing.textCleaner import select_only_keyword, select_only_english, remove_special_char

kaggle_tweets = pd.read_csv('./data/tweets_with_sentiment.csv')
kaggle_tweets['text'] = kaggle_tweets['text'].astype(str)
kaggle_tweets['text'] = kaggle_tweets['text'].str.lower()
print("Kaggle:\n")
print(kaggle_tweets)
real_tweets = pd.read_json('../data/train/tweetsTSLA_with_label.json')
real_tweets = real_tweets.append(pd.read_json('../data/train/tweetsAMZN_with_label.json'), ignore_index=True)
real_tweets.drop(columns=['number_follower', 'number_retweets', 'number_likes', 'number_comments', 'datetime', 'username'], inplace=True)
real_tweets['text'] = real_tweets['text'].str.lower()
real_tweets = select_only_keyword(real_tweets, 'tesla', 'tsla')
real_tweets = select_only_english(real_tweets)
real_tweets['text'] = real_tweets['text'].apply(remove_special_char)
print("Real tweets:\n")
print(real_tweets)

In [ ]:
from xgboost import XGBClassifier

df = kaggle_tweets.append(real_tweets, ignore_index=True)
x_train, x_test, y_train, y_test = train_test_split(df['text'], df['target'], test_size=0.2, random_state=11)

print("Before undersampling: ", Counter(y_train))

# Convert x_train to np_array for rebalance
x_train = x_train.values.reshape(-1, 1)
x_train, y_train = make_imbalance(x_train, y_train,
                                  sampling_strategy={'positive': 491, 'neutral': 491, 'negative': 491},
                                  random_state=0)

# Return to pandas series
x_train = pd.Series(np.squeeze(x_train))
print("After undersampling: ", Counter(y_train))

# Pipeline Classifier
clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', XGBClassifier())   # XGBClassifier è il migliore in acc ma basso positive recall
    # Random Forest Classifier, Logistic Regression e SVC sono buoni (0.75 acc)
    # MultinomialNB e BernoulliNB meno buoni (0.67 acc)
])

# Training the Pipeline Classifier
clf.fit(x_train, y_train)

#Testing of the Pipeline
predicted = clf.predict(x_test)

#Extracting statistics and metrics
accuracy = accuracy_score(predicted, y_test)
print("Accuracy on test set: ", accuracy)
print("Metrics per class on test set:")

print("Confusion matrix:")
metrics.confusion_matrix(y_test, predicted)

print(metrics.classification_report(y_test, predicted, target_names=["positive", "neutral", "negative"]))

# Save the classifier
#joblib.dump(clf, 'sentiment_classifier.pkl')
